In [30]:
import os
import sys
import numpy as np
import pickle
import matplotlib.pyplot as plt
import regex as re
import shutil
from shutil import copytree as copy_tree

In [27]:
pattern = '\d{4}-(0[1-9]|1[0-2])-(0[1-9]|[12][0-9]|3[01])'
p = re.compile(pattern)

In [2]:
exp_dir = '/scratch/shantanu/exps_15022022'
print(os.listdir(exp_dir))

['ans_rgb', 'occant_rgb', 'occant_rgbd', 'occant_maskeddepth', 'postattn_ckdepth', 'basic_transformer', 'occant_chandrakarbev', 'occant_vo_maskeddepth', 'basictransformer_wboundary', 'occant_chandrakardepth', 'crossview', 'occant_depthraw', 'preattn_ckdepth']


In [8]:
exps = ['postattn_ckdepth', 'basic_transformer', 'occant_chandrakarbev', 
        'occant_vo_maskeddepth', 'basictransformer_wboundary', 'occant_chandrakardepth', 
        'crossview', 'occant_depthraw', 'preattn_ckdepth'] #os.listdir(exp_dir)

In [9]:
backup_dir = '/scratch/shantanu/exps_15Feb22_backup'
os.makedirs(backup_dir, exist_ok=True)

In [10]:
backup_weight_range = 3 # number of weights to backup

In [31]:
def backup(exp):
    log_folder = f'{exp_dir}/{exp}/val/step_logs/'
    log_steps = sorted([int(os.path.splitext(x)[0]) for x in 
                        os.listdir(log_folder)])
    val_loss = []
    for step in log_steps:
        log = np.load(os.path.join(f'{exp_dir}/{exp}/val/step_logs/{step}.pkl'), allow_pickle=True)
        val_loss.append(log['loss'])

    sorted_loss_indices = sorted(range(len(val_loss)), key=lambda x: val_loss[x])
    if np.any(['weights_' in x for x in os.listdir(f'{exp_dir}/{exp}')]):
        sub_dir = ''
    else:
        sub_dir = 'models'
    
    weights_dir = os.path.join(exp, sub_dir)
    
    weight_folders = sorted([x for x in os.listdir(f'{exp_dir}/{weights_dir}') 
                             if os.path.isdir(f'{exp_dir}/{weights_dir}/{x}') and 'weights_' in x], 
                            key=lambda x: int(x.split('_')[-1]))
    
    if os.path.exists(os.path.join(backup_dir, exp)):
        shutil.rmtree(os.path.join(backup_dir, exp))
    os.makedirs(os.path.join(backup_dir, exp))

    for idx in range(backup_weight_range):
        tgt_weight_folder = weight_folders[sorted_loss_indices[idx]]
        copy_tree(os.path.join(exp_dir, weights_dir, tgt_weight_folder),
                 os.path.join(backup_dir, weights_dir, f'{idx}_{tgt_weight_folder}'))
        
    
    copy_tree(os.path.join(exp_dir, weights_dir, weight_folders[-1]), 
              os.path.join(backup_dir, weights_dir, f'last_{weight_folders[-1]}'))

    if os.path.exists(os.path.join(exp_dir, exp, 'train')):
        copy_tree(os.path.join(exp_dir, exp, 'train'), os.path.join(backup_dir, exp, 'train'))
    
    for folder in os.listdir(os.path.join(exp_dir, exp)):
        if re.match(p, folder) is not None:
            copy_tree(os.path.join(exp_dir, exp, folder), os.path.join(backup_dir, exp, folder))
        
    copy_tree(os.path.join(exp_dir, exp, 'val'), os.path.join(backup_dir, exp, 'val'))

In [32]:
for exp in exps:
    print(exp)
    backup(exp)

postattn_ckdepth
basic_transformer
occant_chandrakarbev
occant_vo_maskeddepth
basictransformer_wboundary
occant_chandrakardepth
crossview
occant_depthraw
preattn_ckdepth
